In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("train.csv")
data.head()

: 

In [ ]:
data.info()

In [ ]:
object_columns = data.select_dtypes(include=['object']).columns
[print(col_name, ":", data[col].unique()) for col, col_name in zip(data[object_columns], object_columns)]